In [ ]:
from copy import deepcopy
from itertools import chain, count, cycle, repeat
import numpy as np
from IPython.display import clear_output
import sys
import os

In [51]:
SHAPEDEFS = (
    [(0, 0), (0, 1), (0, 2), (0, 3)],
    [(0, 1), (1, 0), (1, 1), (1, 2), (2, 1)],
    [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)],
    [(0, 0), (1, 0), (2, 0), (3, 0)],
    [(0, 0), (0, 1), (1, 0), (1, 1)],
)

def get_shapes():
    return cycle(SHAPEDEFS)

In [ ]:
with open('example_input.txt', 'r') as fd:
    inp = fd.read()

In [ ]:
with open('viz.txt', 'r') as fd:
    viz = fd.read().splitlines()
    viz_rev = '\n'.join(viz[::-1])
    with open('viz_rev.txt', 'w') as fd:
        fd.write(viz_rev)

In [ ]:
viz_rev_lines = viz[::-1]
line_len = len(viz_rev_lines[0]) + 1
num_lines = len(viz_rev_lines)
idx = 0
for i in range(num_lines // 2):
    for j in range(i + 5, num_lines - i + 1):
        chunk = '\n'.join(viz_rev_lines[i:j])
        offset = j * line_len
        if viz_rev.find(chunk, offset) >= 0:
            idx = viz_rev.index(chunk, offset)
            if idx - offset == 0:
                if viz_rev.find(chunk, idx) >= 0:
                    idx2 = viz_rev.index(chunk, idx)
                    if idx2 - idx == 0:
                        print(i, j - i, offset, viz_rev.count(chunk))
            # nidx = viz_rev.index(chunk, offset)
            # cnt = viz_rev.count(chunk)
            # if cnt > 1:
            #     if nidx - j == 0:
            #         print(i, j - i, nidx - j, cnt)

In [ ]:
inp2 = inp + inp[:int(len(inp) // 2)]
size = len(inp2)
half_size = int(size // 2)

In [ ]:
for i in range(half_size):
    for j in range(i, size - i + 1):
        chunk = inp2[i:j]
        cnt = inp2.count(chunk)
        if cnt > 1:
            nidx = inp2.index(chunk, j)
            if nidx - j == 0:
                print(i, j - i, nidx - j, cnt)

In [52]:
def get_input(example=False):
    fname = 'example_input.txt' if example else 'input.txt'
    with open(fname, 'r') as fd:
        return fd.read()

def get_offsets_from_input(inp):
    """Converts the < and > characters in the input to 2D coordinate offsets. Repeats forever."""
    return cycle((0, -1) if c == '<' else (0, 1) for c in inp.strip())

In [47]:
def get_grid_shape(grid):
    return len(grid), len(grid[0]) if grid else 0

def get_absolute_shape_positions(shape):
    x_pos, y_pos = shape['pos']
    for offset in shape['offsets']:
        subshape_x = x_pos + offset[0]
        subshape_y = y_pos + offset[1]
        yield subshape_x, subshape_y

def has_collision(shape, grid):
    rows, cols = get_grid_shape(grid)
    for shape_x, shape_y in get_absolute_shape_positions(shape):
        if shape_x < 0:
            return True
        elif shape_y >= cols or shape_y < 0:
            return True
        elif shape_x < rows and grid[shape_x][shape_y]:
            return True
    return False

def extend_grid(grid, num_rows, fill_value=0, copy=False):
    if copy:
        grid = deepcopy(grid)
    cols = get_grid_shape(grid)[1]
    grid.extend([[fill_value] * cols for _ in range(num_rows)])
    
    return grid

def add_shape_to_grid(shape, grid):
    rows, cols = get_grid_shape(grid)
    for shape_x, shape_y in get_absolute_shape_positions(shape):
        if shape_x + 3 >= rows:
            extend_grid(grid, shape_x - rows + 1)
        grid[shape_x][shape_y] = shape['num']

def get_top_row_with_piece(grid):
    """Highest row on the grid that contains a piece, or -1 if the grid is empty."""
    for r in reversed(range(len(grid))):
        if any(grid[r]):
            return r
    return -1

def visualize_grid(grid, shape=None, filepath=None):
    writer = sys.stdout
    if filepath is not None:
        writer = open(filepath, 'a')
    try:
        shape_coords = list(get_absolute_shape_positions(shape)) if shape else []
        height, width = get_grid_shape(grid)
        for r, row in reversed(list(enumerate(grid))):
            row_out = []
            for c, cell_value in enumerate(row):
                cell = (r, c)
                v = '.'
                if cell in shape_coords:
                    v = '@'
                if cell_value > 0:
                    # Draw collisions with !
                    v = '#'
                    # v = str(cell_value) if v == '.' else '!'
                row_out.append(v)
            writer.write(' '.join([f'{v: <0}' for v in row_out]) + '\n')
        writer.write(f'+-{"--"*(width-2)}+\n\n')
    finally:
        if filepath is not None:
            writer.close()


In [ ]:
for offset in range(100):
    start_idx = 0
    for j in range(offset):
        start_idx = d.index('\n', start_idx + 1)
    idx = start_idx
    print('--line offset', d[:start_idx + 1].count('\n'))
    for i in range(100):
        idx = d.index('\n', idx + 1)
        txt = d[start_idx + 1:idx]
        cnt = d.count(txt)
        if cnt > 1:
            print(i, cnt)
            lines = txt.splitlines()
            if len(lines) > 1 and lines[0] == lines[-1]:
                print('eq', lines[0], '!', lines[-1])

In [71]:
grid = [[0] * 7 for _ in range(6)]
highest_piece = -1
shapes = get_shapes()
inp = get_input(example=False)
movements = chain.from_iterable(zip(get_offsets_from_input(inp), repeat((-1, 0))))
for i in range(2022):
    shape = {'offsets': next(shapes), 'pos': [highest_piece + 4, 2], 'num': i + 1}
    # if i % 50 == 0:
    #     visualize_grid(grid, shape, filepath='viz.txt')

    while True:
        # clear_output()
        # visualize_grid(grid, shape, filepath='viz.txt')

        movement = next(movements)
        test_shape = deepcopy(shape)
        test_shape['pos'] = [shape['pos'][0] + movement[0], shape['pos'][1] + movement[1]]
        if has_collision(test_shape, grid):
            if movement == (-1, 0):
                extend_grid(grid, max((highest_piece + 6) - len(grid), 0))
                add_shape_to_grid(shape, grid)
                highest_piece = get_top_row_with_piece(grid)
                for j in range(shape['pos'][0], len(grid)):
                    nz = sum([1 for c in grid[j] if c > 0])
                    if nz  == 7:
                        print(j)
                break
        else:
            shape = test_shape

if os.path.exists('viz.txt'):
    os.remove('viz.txt')
visualize_grid(grid, shape, filepath='viz.txt')
print(highest_piece + 1)

422
521
663
675
928
956
1039
1078
1115
1256
1257
1284
1603
1605
2050
2263
2281
2617
2878
2887
3069
3159


In [ ]:
grid = [[0] * 7 for _ in range(6)]
highest_piece = -1
shapes = get_shapes()
inp = get_input(example=True)
wait_until_shapes = len(inp) * len(SHAPEDEFS)
movements = chain.from_iterable(zip(get_offsets_from_input(inp), repeat((-1, 0))))
for i in count():
    shape = {'offsets': next(shapes), 'pos': [highest_piece + 4, 2], 'num': i + 1}

    while True:
        movement = next(movements)
        test_shape = deepcopy(shape)
        test_shape['pos'] = [shape['pos'][0] + movement[0], shape['pos'][1] + movement[1]]
        if has_collision(test_shape, grid):
            if movement == (-1, 0):
                extend_grid(grid, max((highest_piece + 6) - len(grid), 0))
                add_shape_to_grid(shape, grid)
                highest_piece = get_top_row_with_piece(grid)
                break
        else:
            shape = test_shape

if os.path.exists('viz.txt'):
    os.remove('viz.txt')
visualize_grid(grid, shape, filepath='viz.txt')
print(highest_piece + 1)

In [ ]:
d, r = divmod(1000000000000, 35)

In [ ]:
r

In [ ]:
(d * 53) + 25

In [ ]:
visualize_grid(grid)